<a href="https://colab.research.google.com/github/SharlotteManganye/Deep-Learning-CNN/blob/main/Sigma_Pi_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class ProductUnitNN(nn.Module):
    def __init__(self, input_dim, hid_dim, output_dim):
        super(ProductUnitNN, self).__init__()
        self.input_dim = input_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        # Initialize weights
        self.w1 = nn.Parameter(torch.randn(input_dim, hid_dim).to(device), requires_grad=True)
        self.w2 = nn.Parameter(torch.randn(hid_dim, output_dim).to(device), requires_grad=True)
        self.bias = nn.Parameter(torch.ones(output_dim) * -1, requires_grad=True).to(device)

    def forward(self, x):

 # FEATURE EXTRACTION (new convolution layer Product Units)
        # Adjust net_sig computation
        net_sig = torch.zeros((x.size(0), self.hid_dim), device=x.device)
        for i in range(self.input_dim):

            condition = x[:, i].unsqueeze(1) != 0
            values_to_add = torch.sum(self.w1 * torch.log(torch.abs(x.unsqueeze(2) - 1)), dim=1)
  # Expand condition to match net_sig's shape for broadcasting
            condition = condition.expand_as(net_sig)  # Becomes (batch_size, hid_dim)
            net_sig = torch.where(condition, values_to_add, net_sig)


        z = net_sig @ self.w2 + self.bias
 #CNN
        x = self.cnn_model(z)  # Feature extraction
        x = x.view(x.size(0), -1)  # Flatten the inputs
        x = self.fc_model(x)  # Fully connected layers
        return  F.log_softmax(x, dim=1)

# Training settings
batch_size = 32
learning_rate = 0.1
epochs = 10

# MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Model, loss function, optimizer
input_dim = 784  # 28x28 images flattened
hid_dim = 100    # Example hidden dimension
output_dim = 10  # 10 classes for MNIST


model = ProductUnitNN(input_dim, hid_dim, output_dim)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.view(-1, input_dim).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Testing the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.view(-1, input_dim).to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


NameError: name 'device' is not defined